In [ ]:
import ROOT as r
import math
from BlindersPy3 import Blinders
from BlindersPy3 import FitType
r.gStyle.SetOptStat(0)
r.gStyle.SetOptFit(1111)

In [ ]:
f = r.TFile("./newPileupHists_gm2offline_ana_60h_pileup_corrected_only.root")
f.cd()
f.ls()

In [ ]:
c = r.TCanvas()
c.Divide(3,1)
c.cd(1)
f.corrected.ProjectionX().Draw() #time (us)
c.cd(2)
f.corrected.ProjectionY().Draw() #energy (MeV)
c.cd(3)
f.corrected.ProjectionZ().Draw() # calonum
c.Draw()

In [ ]:
f.corrected.ProjectionX().FindBin(4)

In [ ]:
c2 = r.TCanvas()
c2.SetLogy()
h = f.corrected.ProjectionX()
h.GetXaxis().SetRangeUser(0,300)
h.Draw()

#get max an min bins for energy cut. Want the bounds to be > 1300 and < 3000 to match last analysis
# could probably get away with > 1000

energyhist = f.corrected.ProjectionY()
minenergybin = energyhist.FindBin(1000)
maxenergybin = energyhist.FindBin(3000)
print(minenergybin,maxenergybin)

h2 = f.corrected.ProjectionX("px_cut",minenergybin,maxenergybin,0,-1,"")
h2.SetLineColor(2)
h2.Draw("SAME")

c2.Draw()

In [ ]:
# blinded instance
getBlinded = Blinders(FitType.Omega_a, 'This is my fight song. Blinding my plot song.')
def blinded_wiggle(x, p):
    norm  = p[0]
    life  = p[1]
    asym  = p[2]
    R     = p[3]
    phi   = p[4]
    
    time  = x[0]
    omega = getBlinded.paramToFreq(R)
#    omega = myBlinder.paramToFreq(R) unblind version
    
    return norm * math.exp(-time/life) * (1 - asym*math.cos(omega*time + phi))

In [ ]:
ding = "full_fit"
func = r.TF1(ding, blinded_wiggle, 30,400,5)
func.SetParNames('N','#tau','A','R','#phi')
func.SetParameters(1500,64.4,0.4,0,0)
func.SetLineColor(2)
func.SetNpx(1000)
r.gStyle.SetOptFit(111)

c = r.TCanvas()
h2.SetLineColor(1)
h2.Draw()
h2.GetXaxis().SetRangeUser(30,500)
h2.Fit(ding,'REM')
func.Draw("SAME")

#h2.GetXaxis().SetRangeUser(30,100)

c.Draw()

In [ ]:
#h2.GetXaxis().SetRangeUser(30,100)
#c.Draw()

In [ ]:
#The class calculates the  difference between the histogram and the fit function at each point 
#    and divides it by the uncertainty.
c2 = r.TCanvas()
rp1 = r.TRatioPlot(h2)
rp1.Draw()
rp1.GetLowerRefYaxis().SetTitle("Residual")
rp1.GetLowerRefYaxis().SetRangeUser(-3,3)
tgraph = rp1.GetCalculationOutputGraph()
dong = r.TGraphAsymmErrors(tgraph)
dong.SetTitle("asym")
c2.Draw()

In [ ]:
c3 = r.TCanvas()
dong.Draw("a p0 l")
dong.GetYaxis().SetRangeUser(-10,10)
c3.Draw()

In [ ]:
ratiolist = []
ratiolist.append(dong)
namelist = ['full']

In [ ]:
residxformlist = []
for j in range(0,len(namelist)):
    c3 = r.TCanvas()
    #tgraph3.Draw()
    tgraph3 = ratiolist[j]
    c3.Draw()
    residualN = tgraph3.GetN() - 2
    listx = []
    listy = []
    minbin = 33
    for i in range(minbin,residualN):
        xi = r.Double(9999)
        yi = r.Double(9999)
        tgraph3.GetPoint(i,xi,yi)
        listx.append(xi)
        listy.append(yi)
    #print(listx)
    
    import numpy
    window = numpy.bartlett(residualN)
    listywindow = []
    for i in range(minbin,residualN-minbin):
        listywindow.append(listy[i])#*window[i])
    fftyreal = numpy.real(numpy.fft.fft(listywindow))
    fftyim = numpy.imag(numpy.fft.fft(listywindow))
    ffty = []
    for i in range(0,len(fftyreal)):
        fftyi = numpy.sqrt(fftyreal[i]**2 + fftyim[i]**2)
        ffty.append(fftyi)
#    print(ffty)

    Npart = residualN - minbin
    minBinCenter = listx[0]
    maxBinCenter = listx[len(listx)-1]
    capT = maxBinCenter - minBinCenter
    print(Npart, capT, minBinCenter, maxBinCenter)
    deltaT = capT/Npart #microseconds
    deltaF = 1/capT
    print(deltaT, deltaF)

    deltaTns = deltaT*1000 #nanoseconds
    limmaxHz = (1/(deltaTns*math.pow(10.0,-9)))
    limmaxMHz = limmaxHz / math.pow(10,6)

    limmax = 2*deltaF*Npart #400-25
    print(limmax,limmaxMHz)

    hresid = r.TH1D("hresid","FFT of Residual from 5-Parameter Fit",residualN,0,residualN)
    for i in range(0,len(ffty)):
        hresid.SetBinContent(i,ffty[i])
    hresid.SetBins(residualN,0,limmaxMHz)
    hresid.GetXaxis().SetRangeUser(0,limmaxMHz/2)
    
    #hresid.Scale(1/(hresid.Integral()))
    
    hresid.Draw("HIST P0 L")
    
    c3.Draw()
    
    residxformlist.append(hresid.DrawClone(namelist[j]+"residxform"))

In [ ]:
c4 = r.TCanvas()
hresid.Draw("P0 L")
hresid.GetXaxis().SetRangeUser(0,2.5)
hresid.GetXaxis().SetTitle("Frequency (MHz)")
hresid.GetYaxis().SetTitle("Arb. Units")
c4.Draw()

In [ ]:
from matplotlib import pyplot as plt
plt.plot(listy[25:])
plt.plot(listywindow[25:])
plt.show

In [ ]:
def peak_fit_bw(x, p):
    return p[0]*r.TMath.BreitWigner(x[0],p[1],p[2]) + p[3]

def peak_fit_simple(x, p):
    return p[0]*r.TMath.Exp(-0.5*((x[0]-p[1])/p[2])*((x[0]-p[1])/p[2])) + p[3]

func = r.TF1("peak1", peak_fit_simple, .35,.4,4)
func.SetParNames('A','Mean','#gamma','+ C')
func.SetParameters(.0001,.23,.1,0)
func.SetParLimits(1,.3,.4)
func.SetLineColor(2)
func.SetNpx(1000)

func2 = r.TF1("pol2func", "pol2", .35,.4,3)
func2.SetParameters(.3,.23,-1)

In [ ]:
c5 = r.TCanvas()
hresid.Draw("P0 L")
hresid.GetXaxis().SetRangeUser(0.3,0.65)
hresid.GetXaxis().SetTitle("Frequency (MHz)")
hresid.GetYaxis().SetTitle("Arb. Units")
hresid.Fit("peak1","REMB")
c5.Draw()